In [2]:
import mysql.connector
import pandas as pd
# import pandasql as ps
import matplotlib.pyplot as plt
# Import argsort
from numpy import argsort
from Functions.NLP.alertp1_nlp import *

In [3]:
creds = pd.read_csv('/Users/gabrielabib/Desktop/NOVA_SBE/PBL/credentials.csv')

In [4]:
#Connection to the database
host = creds['Value'][0]
user = creds['Value'][1]
password = creds['Value'][2]
database = creds['Value'][3]
port = creds['Value'][4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()
#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',)]
1138


In [5]:
alertP1 = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)
#changing columns with date to a datetime
alertP1["DATA_ENVIO"] = pd.to_datetime(alertP1["DATA_ENVIO"],dayfirst=True, yearfirst=False )
alertP1["DATA_RECEPCAO"] = pd.to_datetime(alertP1["DATA_RECEPCAO"],dayfirst=True, yearfirst=False)
alertP1["DATA_RETORNO"] = pd.to_datetime(alertP1[alertP1["DATA_RETORNO"]!='26/06/0214']["DATA_RETORNO"],dayfirst=True, yearfirst=False)
alertP1["DATA_REALIZACAO"] = pd.to_datetime(alertP1["DATA_REALIZACAO"],dayfirst=True, yearfirst=False)
alertP1["DATA_MARCACAO"] = pd.to_datetime(alertP1["DATA_MARCACAO"],dayfirst=True, yearfirst=False)
#change the blank values in proveniencia to unknown
alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'
#create a column with the output of a referral
alertP1['result']=['1' if x in [0,14,15,53,8,12,13] else '0' if x in [1,6,10,7] else '' for x in alertP1['COD_MOTIVO_RECUSA'] ]
#create a list with code of units inside area
area_list=[3150502,3151672,3150671,3150572,3150305,3150571,3151571,3151574,3150371,3151573,3150672,3152401,3150605,3152471,3151575,3151671,3151601,3150573,3151576,3150506,3150504,3152400,3150501,3150603,3151707,9999999,3152403,3151400,3152100,3151401,4021100,4021104,3152002]
#create a column with area of the unit
alertP1['area']=['inside area' if x in area_list else 'outside area' for x in alertP1['COD_UNID_SAUDE_PROV'] ]
#compute length of text
alertP1['text_length']=alertP1['Texto'].str.len()
#step of referral(first or second or ..)
alertP1['step']=alertP1.sort_values(by=['DATA_RECEPCAO']).groupby('ID_DOENTE').cumcount()+1
#speciality 
alertP1['speciality_type'] = ['General Neurology' if x == 'NEUROLOGIA' else 'Other specialities'  for x in alertP1['DES_ESPECIALIDADE']]

alertP1.head()

/Users/gabrielabib/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/var/folders/g5/43zxmsfs73v6lgj3_2k04qj80000gp/T/ipykernel_8077/515315357.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/var/folders/g5/43zxmsfs73v6lgj3_2k04qj80000gp/T/ipykernel_8077/515315357.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PR

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,trata data recusa,resume saída,mês_saida,ano_saida,Texto,result,area,text_length,step,speciality_type
0,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575,USF TORRE DA MARINHA,CS/USF,CTH,3,Normal,...,21/05/2015,12/06/2015,6,2015,"Utente de 75 anos, refere tremor desde há 5 an...",1,inside area,701,1,General Neurology
1,CSAS,LVMVBD,LOHLTRTS,FZFZSDB,3152471,USF SERVIR SAUDE-CS,CS/USF,CTH,,unknown,...,23/04/2012,02/07/2012,7,2012,Avaliação neurologica para avaliação da toma d...,1,inside area,363,1,General Neurology
2,ADCFD,LVMMVJ,LQHROQHL,,0,,OUTRA,SAM,,unknown,...,26/09/2014,13/01/2015,1,2015,cefaleias,1,outside area,9,1,Other specialities
3,SSZDBZ,LVMMLV,LPHQULSU,CSGGHEZ,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,25/10/2013,03/12/2013,12,2013,"Utente do sexo feminino com 30 anos de idade, ...",1,inside area,417,1,General Neurology
4,SEAHEG,LVMNDL,LPHHVHVL,,0,,OUTRA,unknown,,unknown,...,26/02/2013,26/02/2013,2,2013,Sem relatorio clinico,0,outside area,22,1,General Neurology


In [6]:
alertP1_text = alertP1[alertP1['text_length']>0]
alertP1_text

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,trata data recusa,resume saída,mês_saida,ano_saida,Texto,result,area,text_length,step,speciality_type
0,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575,USF TORRE DA MARINHA,CS/USF,CTH,3,Normal,...,21/05/2015,12/06/2015,6,2015,"Utente de 75 anos, refere tremor desde há 5 an...",1,inside area,701,1,General Neurology
1,CSAS,LVMVBD,LOHLTRTS,FZFZSDB,3152471,USF SERVIR SAUDE-CS,CS/USF,CTH,,unknown,...,23/04/2012,02/07/2012,7,2012,Avaliação neurologica para avaliação da toma d...,1,inside area,363,1,General Neurology
2,ADCFD,LVMMVJ,LQHROQHL,,0,,OUTRA,SAM,,unknown,...,26/09/2014,13/01/2015,1,2015,cefaleias,1,outside area,9,1,Other specialities
3,SSZDBZ,LVMMLV,LPHQULSU,CSGGHEZ,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,25/10/2013,03/12/2013,12,2013,"Utente do sexo feminino com 30 anos de idade, ...",1,inside area,417,1,General Neurology
4,SEAHEG,LVMNDL,LPHHVHVL,,0,,OUTRA,unknown,,unknown,...,26/02/2013,26/02/2013,2,2013,Sem relatorio clinico,0,outside area,22,1,General Neurology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,ZAEDSC,MIINNJJJJJ,LRHSSOHP,,0,,OUTRA,SAM,,unknown,...,25/02/2016,27/06/2016,6,2016,Estenose carotidea assintomática bilateral < 7...,1,outside area,164,1,General Neurology
1778,AZGAFH,MIINCIL,LRHLHOOH,,0,,OUTRA,SAM,,unknown,...,23/02/2015,23/02/2015,2,2015,61 anos operada por patologia benigna da mama...,0,outside area,142,1,General Neurology
1779,FGAZCD,MIINCBE,LRHOVHOP,ZHZHASSH,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,14/07/2015,14/07/2015,7,2015,Utente de 69A com Perda de conhecimento a 6.5....,0,inside area,491,1,General Neurology
1780,FGAZCD,MIINCBE,LRHRTLPV,ZHCHBECG,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,3,Normal,...,08/02/2016,08/02/2016,2,2016,Utente de 70A com historia de lipotimia em Mai...,,inside area,225,2,General Neurology


In [7]:
disease_class(alertP1_text, 'Texto')    

/Users/gabrielabib/Desktop/NOVA_SBE/PBL/PBL-HGO/Functions/NLP/alertp1_nlp.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[d] = df[column].str.contains(d)
/Users/gabrielabib/Desktop/NOVA_SBE/PBL/PBL-HGO/Functions/NLP/alertp1_nlp.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[d] = df[column].str.contains(d)
/Users/gabrielabib/Desktop/NOVA_SBE/PBL/PBL-HGO/Functions/NLP/alertp1_nlp.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,step,speciality_type,cefaleia,demência,convulsão,epilepsia,sincope,vertigem,tremor,acidente vascular cerebral
0,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575,USF TORRE DA MARINHA,CS/USF,CTH,3,Normal,...,1,General Neurology,False,False,False,False,False,False,True,False
1,CSAS,LVMVBD,LOHLTRTS,FZFZSDB,3152471,USF SERVIR SAUDE-CS,CS/USF,CTH,,unknown,...,1,General Neurology,False,False,False,False,False,False,False,False
2,ADCFD,LVMMVJ,LQHROQHL,,0,,OUTRA,SAM,,unknown,...,1,Other specialities,True,False,False,False,False,False,False,False
3,SSZDBZ,LVMMLV,LPHQULSU,CSGGHEZ,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,1,General Neurology,True,False,False,False,False,False,False,False
4,SEAHEG,LVMNDL,LPHHVHVL,,0,,OUTRA,unknown,,unknown,...,1,General Neurology,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1777,ZAEDSC,MIINNJJJJJ,LRHSSOHP,,0,,OUTRA,SAM,,unknown,...,1,General Neurology,False,False,False,False,False,False,False,False
1778,AZGAFH,MIINCIL,LRHLHOOH,,0,,OUTRA,SAM,,unknown,...,1,General Neurology,False,False,False,False,False,False,False,False
1779,FGAZCD,MIINCBE,LRHOVHOP,ZHZHASSH,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,1,General Neurology,False,False,False,False,False,False,False,False
1780,FGAZCD,MIINCBE,LRHRTLPV,ZHCHBECG,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,3,Normal,...,2,General Neurology,False,False,False,False,False,False,False,False


In [8]:
cases = alertP1_text.groupby('step')[['cefaleia','demência','convulsão','epilepsia','sincope','vertigem','tremor','acidente vascular cerebral']].sum().reset_index()
cases['total']=cases.sum(axis=1)
cases.iloc[-1,:]=cases.sum(axis=0)
cases

,step,cefaleia,demência,convulsão,epilepsia,sincope,vertigem,tremor,acidente vascular cerebral,total
0,1,139,37,2,65,4,12,116,2,378
1,2,14,11,1,28,1,0,27,1,85
2,3,3,1,0,6,0,3,7,0,23
3,4,2,0,1,2,0,0,2,0,11
4,5,0,0,0,0,0,0,0,0,5
5,21,158,49,4,101,5,15,153,3,509


In [9]:
cefaleia = alertP1_text[alertP1_text['cefaleia']==True][['Texto']]
cefaleia = alertP1_text[alertP1_text['cefaleia'] == True][['Texto']]
demência = alertP1_text[alertP1_text['demência'] == True][['Texto']]
convulsão = alertP1_text[alertP1_text['convulsão'] == True][['Texto']]
epilepsia = alertP1_text[alertP1_text['epilepsia'] == True][['Texto']]
sincope = alertP1_text[alertP1_text['sincope'] == True][['Texto']]
vertigem = alertP1_text[alertP1_text['vertigem'] == True][['Texto']]
tremor = alertP1_text[alertP1_text['tremor'] == True][['Texto']]
acidente = alertP1_text[alertP1_text['acidente vascular cerebral' ] == True][['Texto']]
cefaleia

,Texto
2,cefaleias
3,"Utente do sexo feminino com 30 anos de idade, ..."
5,"Doente do sexo feminino, 54 anos, recorre a su..."
6,"Doente do sexo feminino, 83 anos, com queixas ..."
13,"AP: rinite alergica, HTA, sindrome vertiginoso..."
...,...
1755,Justificação 2 episodios de diplopia com cefal...
1756,2 episodios de diplopia com cefaleia occipital...
1758,A Carolina apresenta enxaquecas há cerca de 9 ...
1760,"Utente de 36 anos de idade,natural de António ..."


In [10]:
509/1773

0.2870840383530739

In [13]:
total_cases = alertP1.groupby('Ano_entrada')[['COD_REFERENCIA']].count().reset_index()
total_cases

,Ano_entrada,COD_REFERENCIA
0,2012,1628
1,2013,1828
2,2014,2127
3,2015,2322
4,2016,2245


In [16]:
alertP1_text.groupby('Ano_entrada')[['COD_REFERENCIA']].count()

,COD_REFERENCIA
Ano_entrada,
2012,279
2013,317
2014,370
2015,441
2016,366


2012    0.160394
2013    0.180099
2014    0.209557
2016    0.221182
2015    0.228768
Name: Ano_entrada, dtype: float64

2012    0.157360
2013    0.178793
2016    0.206430
2014    0.208686
2015    0.248731
Name: Ano_entrada, dtype: float64

In [25]:
alertP1.DATA_RECEPCAO[1]

Timestamp('2012-04-17 00:00:00')

In [27]:
alertP1['year'] = pd.DatetimeIndex(alertP1['DATA_RECEPCAO']).year

In [31]:
alertP1_text['year'] = pd.DatetimeIndex(alertP1_text['DATA_RECEPCAO']).year

/var/folders/g5/43zxmsfs73v6lgj3_2k04qj80000gp/T/ipykernel_8077/1647414874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1_text['year'] = pd.DatetimeIndex(alertP1_text['DATA_RECEPCAO']).year


In [28]:
alertP1.year.value_counts(normalize=True).sort_values()

2012    0.160394
2013    0.180099
2014    0.209557
2016    0.221182
2015    0.228768
Name: year, dtype: float64

In [32]:
alertP1_text.year.value_counts(normalize=True).sort_values()

2012    0.157360
2013    0.178793
2016    0.206430
2014    0.208686
2015    0.248731
Name: year, dtype: float64